<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Agent/blob/main/LlamaIndex/5-Human-in-loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 패키지 설치
!pip install llama-index-core llama-index-utils-workflow llama-index-llms-openai llama-parse llama-index-embeddings-openai llama-index-readers-whisper llama-index-tools-yahoo-finance
!pip install llama-index-tools-tavily-research

In [2]:
from google.colab import userdata
import openai
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key  = os.environ["OPENAI_API_KEY"]
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

In [3]:
from llama_index.llms.openai import OpenAI
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.core.workflow import Context
from llama_index.core.workflow import (
    InputRequiredEvent,
    HumanResponseEvent,
)

llm = OpenAI(model="gpt-4o-mini")

In [4]:
# a tool that performs a dangerous task
async def dangerous_task(ctx: Context) -> str:
    """A dangerous task that requires human confirmation."""

    # emit an event to the external stream to be captured
    ctx.write_event_to_stream(
        InputRequiredEvent(
            prefix="Are you sure you want to proceed? ",
            user_name="Laurie",
        )
    )

    # wait until we see a HumanResponseEvent
    response = await ctx.wait_for_event(
        HumanResponseEvent, requirements={"user_name": "Laurie"}
    )

    # act on the input from the event
    if response.response.strip().lower() == "yes":
        return "Dangerous task completed successfully."
    else:
        return "Dangerous task aborted."

workflow = AgentWorkflow.from_tools_or_functions(
    [dangerous_task],
    llm=llm,
    system_prompt="You are a helpful assistant that can perform dangerous tasks.",
)

In [5]:
async def main():
    handler = workflow.run(user_msg="I want to proceed with the dangerous task.")

    async for event in handler.stream_events():
        # capture InputRequiredEvent
        if isinstance(event, InputRequiredEvent):
            # capture keyboard input
            response = input(event.prefix)
            # send our response back
            handler.ctx.send_event(
                HumanResponseEvent(
                    response=response,
                    user_name=event.user_name,
                )
            )

    response = await handler
    print(str(response))

In [10]:
if __name__ == "__main__":
    import asyncio
    import nest_asyncio  # Import nest_asyncio
    nest_asyncio.apply()  # Apply nest_asyncio patch
    asyncio.run(main())  # Now you can run your async function

Are you sure you want to proceed? proceed
The dangerous task has been aborted. If you would like to proceed with it, please confirm your decision.
